In [1]:
import torch
import torch.nn as nn
from utils import *
from models import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
parser = argparse.ArgumentParser(description='ComHG for link prediction')
parser.add_argument('--float', default=np.float32)
parser.add_argument('--dataset', type=str, default='ogbl-ddi')  # ddi collab ppa citation2
parser.add_argument('--result_appendix', type=str, default='', help="if '', time as appendix")
parser.add_argument('--device', type=str, default='0', help="cpu or gpu id") 
parser.add_argument('--directed', type=bool, default=False)
parser.add_argument('--coalesce', type=bool, default=True, help="whether to coalesce multiple edges between two nodes")
parser.add_argument('--use_weight', type=bool, default=False, help="whether to use edge weight.")
parser.add_argument('--use_val', type=bool, default=False, help="whether to use edges in validation set for testing")
parser.add_argument('--collab_year', type=int, default=2010, help="for ogbl-collab.")

parser.add_argument('--use_feature', type=bool, default=False)
parser.add_argument('--use_node_emb', type=bool, default=True)
parser.add_argument('--use_dist', type=bool, default=False, help="whether to use shortest path distance")
parser.add_argument('--use_cn', type=bool, default=False, help="whether to use common neighbor")
parser.add_argument('--use_aa', type=bool, default=False, help="whether to use Adamic/Adar")
parser.add_argument('--use_ja', type=bool, default=False, help="whether to use Jaccard")
parser.add_argument('--use_ra', type=bool, default=False, help="whether to use Resource Allocation")
parser.add_argument('--use_degree', type=bool, default=False)

parser.add_argument('--max_dist', type=int, default=5)
parser.add_argument('--max_cn', type=int, default=100)
parser.add_argument('--max_aa', type=int, default=100)
parser.add_argument('--max_ja', type=int, default=100)
parser.add_argument('--max_ra', type=int, default=20)
parser.add_argument('--max_degree', type=int, default=1000)
parser.add_argument('--mag_ja', type=int, default=100)
parser.add_argument('--mag_aa', type=int, default=10)
parser.add_argument('--mag_ra', type=int, default=10)

parser.add_argument('--heurisctic_reproduce', type=bool, default=False, help="whether to re-produce heuristics, False would speed up training but may decrease the performance")
parser.add_argument('--heurisctic_batch_size', type=int, default=100, help="number of rows in distance computation")
parser.add_argument('--heurisctic_directed', type=bool, default=False, help="use directed graph in distance computation")
parser.add_argument('--heurisctic_reuse', type=bool, default=True)
parser.add_argument('--neg_size', type=float, default=2000, help="size of data_neg_train: neg_size * len(split_edge['train']['edge'])")

parser.add_argument('--adj_hop', type=int, default=2, help="hieghest hop of adj for GNN.")
parser.add_argument('--adj_neg', type=float, default=0.0, help="neg samples (global neighbors) used in adj. ")
parser.add_argument('--adj_neg_dist', type=int, default=10, help="distance from global neighbors to the central node.")
parser.add_argument('--adj_weight', type=str, default='same', help="the method for weights in the original adj. options: same or decay")

parser.add_argument('--atten_type', type=str, default='Multiply',
                    help="attention mechnism. Options: Concat, Cosine, Multiply, no_atten, with repsect to GAT, AGNN, Transformer and GCN")
parser.add_argument('--atten_combine', type=str, default='plus',
                    help="type of combining original adj with attention (if use). must be: plus, multiply or only_atten")
parser.add_argument('--bias', type=bool, default=True)
parser.add_argument('--dim_node_emb', type=int, default=512)
parser.add_argument('--dim_encoding', type=int, default=32, help="dim for encoding heuristics, etc.")
parser.add_argument('--dim_atten', type=int, default=8, help="dim for matrix multiplication. Should be small for large graph")
parser.add_argument('--dim_hidden', type=int, default=None, help="if None, dim_hidden = dim_in")
parser.add_argument('--n_layers', type=int, default=2, help="CNN layers")
parser.add_argument('--n_heads', type=int, default=4, help="multiply head CNN")
parser.add_argument('--n_layers_mlp', type=int, default=5)
parser.add_argument('--residual', type=bool, default=True, help="whether to use residual connection in CNN")
parser.add_argument('--reduce', type=str, default='add', help="combine outputs of multi-head CNN modules. options: concat, add")
parser.add_argument('--negative_slope', type=float, default=0.2, help="negative_slope for leaky_relu")

parser.add_argument('--num_workers', type=int, default=64)
parser.add_argument('--optimizer', type=str, default='Adam', help="'Adam', 'AdamW', 'SGD'")
parser.add_argument('--clip_grad_norm', type=float, default=1.0, help="whether to use clip_grad_norm_ in training")
parser.add_argument('--use_layer_norm', type=bool, default=False, help="whether to use layer norm")
parser.add_argument('--dropout', type=float, default=0.25)
parser.add_argument('--dropout_adj', type=float, default=0.)
parser.add_argument('--lr', type=float, default=0.002)
parser.add_argument('--lr_mini', type=float, default=0.00001, help="lr stops decreasing at lr_mini")
parser.add_argument('--scheduler_gamma', type=float, default=0.997)
parser.add_argument('--shuffle', type=bool, default=True)

parser.add_argument('--runs', type=int, default=1)
parser.add_argument('--epochs', type=int, default=1000)
parser.add_argument('--eval_epoch', type=int, default=1)
parser.add_argument('--batch_size', type=int, default=100000)
parser.add_argument('--batch_num', type=int, default=1000, help="number of batches trained in an epoch")

parser = add_hierarchical_args(parser)
args = parser.parse_args([])  # Empty list since we're in a notebook
args.device = device
args.max_dist = max(args.max_dist, 3) if args.use_dist else 3

In [3]:
# Dataset loading and preprocessing
dataset = PygLinkPropPredDataset(name=args.dataset)
split_edge = dataset.get_edge_split()
data = dataset[0]

args.eval_metrics = 'Hits@20'
args.hitK = [20]
args.dense_sparse = 'dense'

# Set up results directory
if args.result_appendix == '':
    args.result_appendix = '_' + time.strftime("%Y%m%d%H%M%S")
args.dir_result = os.path.join('results/{}{}'.format(args.dataset, args.result_appendix))
print('Results will be saved in ' + args.dir_result)

if not os.path.exists(args.dir_result):
    os.makedirs(args.dir_result)

# Copy source files to results directory
for file in ['main_pred.py', 'utils.py', 'models.py']:
    copy(file, args.dir_result)

# Set up logging
loggers = get_loggers(args)
log_file = osp.join(args.dir_result, 'log.log')

# Log arguments
with open(log_file, 'w') as f:
    print(str(args), file=f)
    print(str(args), file=sys.stdout)

Results will be saved in results/ogbl-ddi_20241212200144
Namespace(float=<class 'numpy.float32'>, dataset='ogbl-ddi', result_appendix='_20241212200144', device=device(type='cuda'), directed=False, coalesce=True, use_weight=False, use_val=False, collab_year=2010, use_feature=False, use_node_emb=True, use_dist=False, use_cn=False, use_aa=False, use_ja=False, use_ra=False, use_degree=False, max_dist=3, max_cn=100, max_aa=100, max_ja=100, max_ra=20, max_degree=1000, mag_ja=100, mag_aa=10, mag_ra=10, heurisctic_reproduce=False, heurisctic_batch_size=100, heurisctic_directed=False, heurisctic_reuse=True, neg_size=2000, adj_hop=2, adj_neg=0.0, adj_neg_dist=10, adj_weight='same', atten_type='Multiply', atten_combine='plus', bias=True, dim_node_emb=512, dim_encoding=32, dim_atten=8, dim_hidden=None, n_layers=2, n_heads=4, n_layers_mlp=5, residual=True, reduce='add', negative_slope=0.2, num_workers=64, optimizer='Adam', clip_grad_norm=1.0, use_layer_norm=False, dropout=0.25, dropout_adj=0.0, lr=

In [4]:
data_pos_train = graph_prepare(args, posneg_split='pos_train')
data_pos_valid = graph_prepare(args, posneg_split='pos_valid')
data_neg_valid = graph_prepare(args, posneg_split='neg_valid')
data_pos_test = graph_prepare(args, posneg_split='pos_test')
data_neg_test = graph_prepare(args, posneg_split='neg_test')


remove non-shortest distance neighbors in high adjs ...
saving adj_degree data to dataset/ogbl_ddi/processed/adj_hop2_neg0.0.pt
finish loading data_pos_train
finish loading data_pos_valid
finish loading data_neg_valid
finish loading data_pos_test
finish loading data_neg_test


In [5]:
def get_dim_in(args, data):
    if data.x != None:
        x_size = data.x.size(-1)
        if args.dataset == 'ogbl-ppa':
            x_size = args.dim_encoding
    args.dim_in = 0
    if args.use_feature and data.x != None:
        args.dim_in += x_size
    if args.use_node_emb:
        args.dim_in += args.dim_node_emb
    if args.use_degree:
        args.dim_in += args.dim_encoding
    if args.dim_in == 0:
        args.dim_in = 1

    return args

In [6]:
def my_DataLoader(args, data, shuffle=False):
    edges = data.edges
    num_edge = edges.size(0)
    perm_size = int(min(args.batch_num * args.batch_size, num_edge))
    if shuffle:
        if num_edge > 1E8:
            perm = np.array(random.sample(range(num_edge), perm_size))
        else:
            perm = np.random.permutation(num_edge)
            perm = perm[:perm_size]
        edges = edges[perm]
    step, end = args.batch_size, perm_size
    perms = [np.array(range(i, i + step)) if i + step < end else np.array(range(i, end)) for i in range(0, end, step)]

    return edges, perms

In [7]:
def train(args, predictor, optimizer, scheduler, data_pos, data_neg):
    #
    predictor.train()
    running_loss = running_examples = 0

    pos_edges, pos_perms = my_DataLoader(args, data_pos, shuffle=args.shuffle)
    neg_edges, neg_perms = my_DataLoader(args, data_neg, shuffle=args.shuffle)
    leniter = min(len(pos_perms), len(neg_perms))
    for i in range(leniter):
        optimizer.zero_grad()
        edge_pos = pos_edges[pos_perms[i]]
        edge_neg = neg_edges[neg_perms[i]]
        y_pos = predictor(data_pos, edge_pos)
        y_neg = predictor(data_neg, edge_neg)
        # avoid imbalance
        if len(y_pos) != len(y_neg):
            break

        pos_loss = (-torch.log(y_pos + 1e-15)).mean()
        neg_loss = (-torch.log(1.0 - y_neg + 1e-15)).mean()
        loss = pos_loss + neg_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), args.clip_grad_norm)#, error_if_nonfinite=True
        optimizer.step()
        # scheduler.step()

        running_loss += loss.item()
        running_examples += 1
    return running_loss / running_examples

In [8]:
@torch.no_grad()
def test(args, predictor, data_pos, data_neg):
    predictor.eval()

    def get_predictions(args, predictor, data_pred):
        pred = []
        edges = data_pred.edges
        num_edge = edges.size(0)
        step, end = args.batch_size*100, num_edge
        perms = [np.array(range(i, i + step)) if i + step < end else np.array(range(i, end)) for i in range(0, end, step)]
        for perm in perms:
            edge_batch = edges[perm]
            y = predictor(data_pred, edge_batch)
            pred.append(y.detach().cpu().squeeze(1))
        return torch.cat(pred, dim=0)

    pred = torch.cat([get_predictions(args, predictor,data_pos), get_predictions(args, predictor,data_neg)], dim=0)
    true = torch.cat([torch.ones(data_pos.edges.size(0)), torch.zeros(data_neg.edges.size(0))], dim=0)

    return pred, true

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
data_neg_train = graph_prepare(args, posneg_split='neg_train')

# tensorboard_writer = SummaryWriter(osp.join(args.dir_result, f'log_{run}.log'))
args = get_dim_in(args, data_pos_train)
predictor = HierarchicalComHG(args).to(args.device)
print('count_parameters:', count_parameters(predictor))
optimizer = get_optimizer(args, predictor.parameters())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=args.scheduler_gamma)


negative sampling ...
negative sampling finished
finish loading data_neg_train
count_parameters: 8896994


In [10]:

for epoch in range(args.epochs):
    data_neg_train.resample_neg_edges()

    loss = train(args, predictor, optimizer, scheduler, data_pos_train, data_neg_train)
    lr_now = optimizer.param_groups[0]["lr"]
    if lr_now > args.lr_mini:
        scheduler.step()

    if epoch % args.eval_epoch == args.eval_epoch - 1:
        # with Timing(name='Times of validation: '):
        val_pred, val_true = test(args, predictor, data_pos_valid, data_neg_valid)
        test_pred, test_true = test(args, predictor, data_pos_test, data_neg_test)
        results = get_eval_result(args, val_pred, val_true, test_pred, test_true)
        for key, result in results.items():
            loggers[key].add_result(result)
            valid_res, test_res = result
            to_print = (f'learning rate: {lr_now:.7f}' + '\n'
                        + f': {key}: Epoch: {epoch:02d}, '
                        + f'Loss: {loss:.4f}, Valid: {100 * valid_res:.2f}%, '
                        + f'Test: {100 * test_res:.2f}%')
            # tensorboard_writer.add_scalar('eval/loss', loss, epoch + 1)
            # tensorboard_writer.add_scalar('eval/Valid', valid_res, epoch + 1)
            # tensorboard_writer.add_scalar('eval/Test', test_res, epoch + 1)
            with open(log_file, 'a') as f:
                print(to_print, file=f)
                print(to_print)


for key in loggers.keys():
    to_print = key
    with open(log_file, 'a') as f:
        print(to_print, file=f)
        loggers[key].print_statistics(f=f)
        print(to_print)
        loggers[key].print_statistics()

learning rate: 0.0020000
: Hits@20: Epoch: 00, Loss: 1.4064, Valid: 0.00%, Test: 0.00%
learning rate: 0.0019940
: Hits@20: Epoch: 01, Loss: 1.3706, Valid: 0.79%, Test: 0.71%
learning rate: 0.0019880
: Hits@20: Epoch: 02, Loss: 0.9284, Valid: 2.00%, Test: 2.96%
learning rate: 0.0019821
: Hits@20: Epoch: 03, Loss: 0.8855, Valid: 1.76%, Test: 1.70%
learning rate: 0.0019761
: Hits@20: Epoch: 04, Loss: 0.8662, Valid: 1.42%, Test: 1.24%
learning rate: 0.0019702
: Hits@20: Epoch: 05, Loss: 0.8338, Valid: 3.98%, Test: 2.53%
learning rate: 0.0019643
: Hits@20: Epoch: 06, Loss: 0.7766, Valid: 4.78%, Test: 3.15%
learning rate: 0.0019584
: Hits@20: Epoch: 07, Loss: 0.6996, Valid: 4.86%, Test: 6.97%
learning rate: 0.0019525
: Hits@20: Epoch: 08, Loss: 0.6500, Valid: 8.67%, Test: 3.83%
learning rate: 0.0019466
: Hits@20: Epoch: 09, Loss: 0.6269, Valid: 9.95%, Test: 5.85%
learning rate: 0.0019408
: Hits@20: Epoch: 10, Loss: 0.6076, Valid: 9.90%, Test: 2.34%
learning rate: 0.0019350
: Hits@20: Epoch: 